In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [ ]:
engine = create_engine(connstr)
PK_FORM = '2021-Q4-BOD2201'
Acronym='GSA'
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [ ]:
query = f""" \
    SELECT TOP 5000 * FROM vwQuestions WHERE PK_FORM LIKE '%{PK_FORM}%'
"""  
df_vwQuestions = get_df(query)

query = f""" \
    SELECT TOP 5000 * FROM  vwOrgSubToComponent WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwOrgSubToComponent = get_df(query)

query = f""" \
    SELECT TOP 5000 * FROM  vwMetricsAnswers WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwMetricsAnswers = get_df(query) 

In [ ]:
for i, r in df_vwQuestions.iterrows():
    s = ''
    s = s + f" -- {str(r['PK'])}  {str(r['QGROUP'])}"
    print( s ) 
 

In [ ]:
###################################################################################

In [ ]:
query = """ \
SELECT TOP 5000 
* 
FROM CVEUnremediated
WHERE PK_OrgSubmission = 29150 
"""   
df_CVEUN = get_df(query)
 

df = pd.read_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_VALID.xlsx').fillna('')
df['Unremediated Count Justification'] = 'Justification' 
df.columns = [c.replace(' ' , ' ') for c in df.columns] 
df.loc[df.CVE=='CVE-2011-3544', 'Unremediated Count Justification'] = ''
df.loc[df.CVE=='CVE-2011-3544', 'Total Unremediated Assets'] = '9999999'
df.columns
df.to_excel(r'C:\Users\Tim\Downloads\BOD2201_2021_2_INVALID.xlsx', index=False) 
df

df.to_sql('CVEUnremediated_UPLOAD', engine, index=True, if_exists='replace')

conn = engine.connect() 
conn.execute('DROP TABLE CVEUnremediated_UPLOAD;')
conn.close()